In [1]:
%load_ext autoreload
%autoreload 2

import IPython
from pathlib import Path
import os
locals = IPython.extract_module_locals() # type: ignore
notebook_name = "/".join(locals[1]["__vsc_ipynb_file__"].split("/"))
os.chdir(Path(notebook_name).parent.parent)

In [2]:
from pathlib import Path

import numpy as np
import polars as pl
import pandas as pd

In [4]:
# Read csv
relations = pl.read_csv(".data/steam/base/recommendations.csv", try_parse_dates=True)
users = pl.read_csv(".data/steam/base/users.csv")
games = pl.read_csv(".data/steam/base/games.csv")

In [35]:
relations = relations.filter(pl.col("is_recommended") == True)
filtered_users = relations.group_by("user_id").agg(pl.len()).filter(pl.col("len") >= 5).select("user_id")
relations = relations.join(filtered_users, on="user_id", how="right")
relations = relations.sort(by="date")

In [36]:
users = filtered_users

In [43]:
users = users.sort("user_id").with_columns(customer_id_map=pl.arange(0, len(users), 1))

user_id,customer_id_map
i64,i64
0,0
13,1
14,2
36,3
45,4
…,…
14306001,1624061
14306002,1624062
14306011,1624063


In [51]:
filtered_apps = relations.select("app_id").unique()
games = games.join(filtered_apps, on="app_id", how="right")
games = games.sort("app_id")
games = games.with_columns(article_id_map=pl.arange(0, len(games), 1))

In [7]:
# Write files to parquet
users.write_parquet(".data/intermediate/users.parquet")
articles.write_parquet(".data/intermediate/articles.parquet")
relations.write_parquet(".data/intermediate/relations.parquet")